In [25]:
import pandas as pd

from collections import Counter

In [26]:
filename = "melon_chart.csv"
raw_data = pd.read_csv(filename)

In [27]:
raw_data.head()


,rank,title,singer,album,date,like
0,1,Perfect Night,Perfect Night,LE SSERAFIM (르세라핌),2024.01.01 ~ 2024.01.07,114126
1,2,Drama,Drama - The 4th Mini Album,aespa,2024.01.01 ~ 2024.01.07,85715
2,3,To. X,To. X - The 5th Mini Album,태연 (TAEYEON),2024.01.01 ~ 2024.01.07,143705
3,4,비의 랩소디,비의 랩소디,임재현,2024.01.01 ~ 2024.01.07,68730
4,5,첫 눈,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',EXO,2024.01.01 ~ 2024.01.07,287039


In [28]:
# 해당 부분은 현재 해결하여 신규 데이터부터 적용할 필요없음.
raw_data['tmp'] = raw_data['album']
raw_data['album'] = raw_data['singer']
raw_data['singer'] = raw_data['tmp']
raw_data.drop(['tmp'],axis=1,inplace = True)
raw_data.head()

,rank,title,singer,album,date,like
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01 ~ 2024.01.07,114126
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01 ~ 2024.01.07,85715
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01 ~ 2024.01.07,143705
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01 ~ 2024.01.07,68730
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01 ~ 2024.01.07,287039


# 1. 연도-계절 별 차트 등장회수 TOP5 조회

In [29]:
# 1. 봄 / 겨울 구분 컬럼 추가
# 봄 : 0 겨울 : 1 
data = raw_data
data['start_date'] = data['date'].apply(lambda x : x.split('~')[0].strip())
data['start_date'] = pd.to_datetime(data['start_date'])
data['month'] = data['start_date'].dt.month
data['year'] = data['start_date'].dt.year
data['season'] = data['month'].apply(lambda x : 1 if x in [1,2,12] else 0)
data.drop(['start_date'],axis=1,inplace = True)

data.head()


,rank,title,singer,album,date,like,month,year,season
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01 ~ 2024.01.07,114126,1,2024,1
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01 ~ 2024.01.07,85715,1,2024,1
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01 ~ 2024.01.07,143705,1,2024,1
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01 ~ 2024.01.07,68730,1,2024,1
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01 ~ 2024.01.07,287039,1,2024,1


In [30]:
# 연도별 계절에 따른 가장 많이 등장한 title,singer,date 
# date ???

# 결과를 저장할 빈 데이터 프레임을 생성
columns=['year', 'season','title', 'singer', 'count']
result_df = pd.DataFrame(columns=columns)

temp = []

# grouped_data = data.groupby(['year','season','title','singer']).size().reset_index(name='count')
grouped_data = data.groupby(['year','season'])

for name,group in grouped_data:
    year,season = name

    # 노래 제목과 가수로 튜플로 만들어 등장회수 세기.
    song_counter = Counter(zip(group['title'],group['singer'])) # {(노래1,가수1):2,(노래2,가수1):1, ....}
    most_song = song_counter.most_common(5) # 상위 5개 [((노래1,가수1),2).... ]

    for (title,singer),count in most_song:
        temp_df = pd.DataFrame(
            {
                'year' : [year],
                'season' : [season],
                'title' : [title],
                'singer' : [singer],
                'count' : [count]
            }
        )
        temp.append(temp_df)
result_df = pd.concat(temp,ignore_index=True)
        



In [31]:
result_df

,year,season,title,singer,count
0,2010,0,A,이효리,30
1,2010,0,너 때문에 미쳐,티아라,12
2,2010,0,루팡 (Lupin),카라,12
3,2010,0,Bubble Love,"MC몽, 서인영",12
4,2010,0,주변인,아웃사이더,12
...,...,...,...,...,...
145,2024,1,Perfect Night,LE SSERAFIM (르세라핌),9
146,2024,1,Drama,aespa,9
147,2024,1,To. X,태연 (TAEYEON),9
148,2024,1,비의 랩소디,임재현,9


# 2. 3년 이상 연속으로 등장한 노래 추출

In [32]:
raw_data.head()

,rank,title,singer,album,date,like,month,year,season
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01 ~ 2024.01.07,114126,1,2024,1
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01 ~ 2024.01.07,85715,1,2024,1
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01 ~ 2024.01.07,143705,1,2024,1
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01 ~ 2024.01.07,68730,1,2024,1
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01 ~ 2024.01.07,287039,1,2024,1


In [33]:
def is_consecutive(years_set):
    sorted_years = sorted(years_set) # 연도 오름차순 정렬
    consecutive_count = 1 # 최소 1년은 등장하므로 1로 초기화.
    max_consecutive = 1 # 연속연도 count 변수

    for i in range(len(sorted_years)-1):
        # 현재연도와 다음연도가 연속적인지 확인. 2023+1 == 2024 
        if sorted_years[i] + 1 == sorted_years[i + 1] :
            consecutive_count += 1
            max_consecutive = max(max_consecutive,consecutive_count)
        else : 
            consecutive_count = 1
    return max_consecutive >= 3 # 3년연속 등장여부

In [34]:
# 연도를 기준으로 등장 연도 조회 ( 중복제거를 위해 set 사용 )
data = raw_data
grouped_data = data.groupby(['title','singer'])['year'].apply(set).reset_index()

In [35]:
# 3년이상 연속출현 여부 , 1 : 출현 0 : 미출현
grouped_data['more_than_3'] = grouped_data['year'].apply(lambda x : 1 if is_consecutive(x) else 0 )

In [36]:
# top100 차트인 했던 연도수 리스트컬럼 
grouped_data['year'] = grouped_data['year'].apply(lambda x :list(x))

In [37]:
result_df = grouped_data.rename(columns = {'year':'years'})

In [38]:
result_df.loc[result_df.more_than_3 == 1]

,title,singer,years,more_than_3
38,2002,Anne-Marie,"[2019, 2020, 2021]",1
79,A,Maroon 5,"[2016, 2017, 2018, 2011, 2012, 2014, 2015]",1
108,A,이해리 (다비치),"[2012, 2013, 2014]",1
115,A bientot,임영웅,"[2024, 2022, 2023]",1
131,After LIKE,IVE (아이브),"[2024, 2022, 2023]",1
...,...,...,...,...
4696,크리스마스니까,"성시경, 박효신, 이석훈, 서인국, VIXX (빅스)","[2016, 2017, 2018, 2019, 2020, 2021, 2022, 202...",1
4753,피 땀 눈물,방탄소년단,"[2016, 2017, 2018]",1
4820,한숨,이하이,"[2016, 2017, 2018]",1
4838,해요 (2022),#안녕,"[2024, 2022, 2023]",1


In [39]:
raw_data.loc[(raw_data['title']=='한숨')&(raw_data['singer']=='이하이')]

,rank,title,singer,album,date,like,month,year,season
14532,33,한숨,이하이,SEOULITE,2018.01.01 ~ 2018.01.07,287328,1,2018,1
14633,34,한숨,이하이,SEOULITE,2018.01.08 ~ 2018.01.14,287329,1,2018,1
14733,34,한숨,이하이,SEOULITE,2018.01.15 ~ 2018.01.21,287329,1,2018,1
14844,45,한숨,이하이,SEOULITE,2018.01.22 ~ 2018.01.28,287329,1,2018,1
14951,52,한숨,이하이,SEOULITE,2018.01.29 ~ 2018.02.04,287329,1,2018,1
15057,58,한숨,이하이,SEOULITE,2018.02.05 ~ 2018.02.11,287329,2,2018,1
15165,66,한숨,이하이,SEOULITE,2018.02.12 ~ 2018.02.18,287329,2,2018,1
15269,70,한숨,이하이,SEOULITE,2018.02.19 ~ 2018.02.25,287329,2,2018,1
15372,73,한숨,이하이,SEOULITE,2018.02.26 ~ 2018.03.04,287329,2,2018,1
15486,87,한숨,이하이,SEOULITE,2018.03.05 ~ 2018.03.11,287329,3,2018,0


# 3. 각 노래별 최초 등장일 조회

In [40]:
data.head()

,rank,title,singer,album,date,like,month,year,season
0,1,Perfect Night,LE SSERAFIM (르세라핌),Perfect Night,2024.01.01 ~ 2024.01.07,114126,1,2024,1
1,2,Drama,aespa,Drama - The 4th Mini Album,2024.01.01 ~ 2024.01.07,85715,1,2024,1
2,3,To. X,태연 (TAEYEON),To. X - The 5th Mini Album,2024.01.01 ~ 2024.01.07,143705,1,2024,1
3,4,비의 랩소디,임재현,비의 랩소디,2024.01.01 ~ 2024.01.07,68730,1,2024,1
4,5,첫 눈,EXO,겨울 스페셜 앨범 '12월의 기적 (Miracles In December)',2024.01.01 ~ 2024.01.07,287039,1,2024,1


In [42]:
data = raw_data
data['start_date'] = data['date'].apply(lambda x : x.split('~')[0].strip())
data['start_date'] = pd.to_datetime(data['start_date'])

result_df = data.groupby(['title','singer'])['start_date'].min().reset_index()

In [43]:
result_df

,title,singer,start_date
0,#결별,"길구봉구, 박보람",2018-04-16
1,#첫사랑,볼빨간사춘기,2018-01-08
2,%% (응응),Apink (에이핑크),2019-01-07
3,...하고 싶다,V.One,2010-03-07
4,..IS YOU,XIA (준수),2016-05-16
...,...,...,...
4961,히히하헤호,"마마무 (Mamamoo), 긱스 (Geeks)",2014-05-26
4962,힐링이 필요해,로이킴,2012-12-03
4963,힘든 건 사랑이 아니다,임창정,2020-11-30
4964,"힘을 내요, 그대",더필름,2011-01-02
